<a href="https://colab.research.google.com/github/DaverArenas/StockExchange_Predictions/blob/main/EDA%2C_Prediction%2C_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Descargamos los datos proporcionados por la competencia usando la API de Kaggle

https://www.kaggle.com/code/unokensuke/eng-eda-prediction-evaluation/notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/_Credentials/kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c jpx-tokyo-stock-exchange-prediction

 98% 236M/241M [00:05<00:00, 45.7MB/s]
100% 241M/241M [00:05<00:00, 45.8MB/s]


In [ ]:
!unzip /content/jpx-tokyo-stock-exchange-prediction.zip -d "/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions"

##2. Importamos los paquetes y librerias necesarias

In [16]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")

##3. Cargamos los datos

In [17]:
prices = pd.read_csv("/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/supplemental_files/stock_prices.csv", parse_dates=["Date"])

In [18]:
prices

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False,-0.003263
1,20211206_1332,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False,-0.008993
2,20211206_1333,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False,-0.009963
3,20211206_1375,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False,-0.015032
4,20211206_1376,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False,0.002867
...,...,...,...,...,...,...,...,...,...,...,...,...
269876,20220624_9990,2022-06-24,9990,576.0,576.0,563.0,564.0,24200,1.0,NaN,False,0.027073
269877,20220624_9991,2022-06-24,9991,810.0,815.0,804.0,815.0,8700,1.0,NaN,False,0.001220
269878,20220624_9993,2022-06-24,9993,1548.0,1548.0,1497.0,1497.0,12600,1.0,NaN,False,0.001329
269879,20220624_9994,2022-06-24,9994,2507.0,2527.0,2498.0,2527.0,7300,1.0,NaN,False,0.003185


##4. Exploratory Data Analysis

###Check the number of missing values per column

In [19]:
prices.isnull().sum()

RowId                    0
Date                     0
SecuritiesCode           0
Open                   818
High                   818
Low                    818
Close                  818
Volume                   0
AdjustmentFactor         0
ExpectedDividend    267870
SupervisionFlag          0
Target                   8
dtype: int64

In [20]:
print((prices["Open"].isnull() == (prices["Volume"]==0)).all())
print((prices["High"].isnull() == (prices["Volume"]==0)).all())
print((prices["Low"].isnull() == (prices["Volume"]==0)).all())
print((prices["Close"].isnull() == (prices["Volume"]==0)).all())

True
True
True
True


In [21]:
prices[(~prices["ExpectedDividend"].isnull())]["Date"].value_counts()

2022-03-28    1404
2021-12-27     327
2022-02-22     130
2022-05-26      48
2022-04-25      43
2022-01-26      34
2022-02-15       7
2022-03-15       5
2022-05-17       5
2021-12-15       3
2022-01-17       2
2022-05-10       2
2022-03-10       1
Name: Date, dtype: int64

In [22]:
prices["SecuritiesCode"].value_counts().sort_values()

1413     94
8806     95
4699    114
2729    118
3157    135
       ... 
9994    135
9900    135
9896    135
9974    135
6201    135
Name: SecuritiesCode, Length: 2000, dtype: int64

In [23]:
pd.pivot(prices, index="Date", columns="SecuritiesCode", values="Volume")[[1413, 8806, 4699, 1375]].tail(30)

SecuritiesCode,1413,8806,4699,1375
Date,,,,
2022-05-16,NaN,NaN,200.0,315200.0
2022-05-17,NaN,NaN,300.0,207600.0
2022-05-18,NaN,NaN,2900.0,282900.0
2022-05-19,NaN,NaN,800.0,249700.0
2022-05-20,NaN,NaN,5300.0,268800.0
2022-05-23,NaN,NaN,900.0,248100.0
2022-05-24,NaN,NaN,0.0,330200.0
2022-05-25,NaN,NaN,3100.0,230300.0
2022-05-26,NaN,NaN,3000.0,228700.0


In [24]:
def MA(series, window=25):
    return series.rolling(window, min_periods=1).mean()

def DMA(series, window=25):
    return series/MA(series, window) - 1

def divergence(series, window=25):
    std = series.rolling(window,min_periods=1).std()
    mean = series.rolling(window,min_periods=1).mean()
    return (series-mean) / std    

def rsi(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:s[s>0].sum()/abs(s).sum())

def stochastic(series, k=14, n=3, m=3):
    _min = series.rolling(k).min()
    _max = series.rolling(k).max()
    _k = (series - _min)/(_max - _min)
    _d1 = _k.rolling(n).mean()
    _d2 = _d1.rolling(m).mean()
    return pd.DataFrame({
                    "%K":_k,
                    "FAST-%D":_d1,
                    "SLOW-%D":_d2,
                    },index=series.index)
    # return _k, _d1, _d2

def psy(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:(s>=0).mean())

def ICH(series):
    conv = series.rolling(9).apply(lambda s:(s.max()+s.min())/2)
    base = series.rolling(26).apply(lambda s:(s.max()+s.min())/2)
    pre1 = ((conv + base)/2).shift(25)
    pre2 = d.Close_adj.rolling(52).apply(lambda s:(s.max()+s.min())/2).shift(25)
    lagg = d.Close_adj.shift(25)
    return conv, base, pre1, pre2, lagg

def roc(series, window=14):
    return series/series.shift(window) - 1

class FeatureBase():
    def create_feature(self, d):
        assert False, "NotImplemented"
        
class MAFeature(FeatureBase):
    def create_feature(self, d):
        return self._create_feature(d["Close_adj"])

    def _create_feature(self, series, window1=5, window2=25):
        ma1 = MA(series, window1).rename("MA1")
        ma2 = MA(series, window2).rename("MA2")
        diff = ma1 - ma2
        cross = pd.Series(
                        np.where((diff>0) & (diff<0).shift().fillna(False), 1,
                            np.where((diff<0) & (diff>0).shift().fillna(False), -1, 0
                                )
                        ),
                        index = series.index, name="MA_Cross"
                )
        return pd.concat([ma1, ma2, cross], axis=1)

In [25]:
def holiday(d):
    return pd.DataFrame({
        "before_holiday":(d["Date"] != d["Date"].shift(-1) - datetime.timedelta(days=1)) | (d["weekday"]==4),
        "after_holiday":(d["Date"] != d["Date"].shift(1) + datetime.timedelta(days=1)) | (d["weekday"]==0)
    }, index=d.index)
def make_features(df):
    df = df[[
        "Date","SecuritiesCode","Open","Close","AdjustmentFactor",
        "Volume"
    ]].copy()
    df["weekday"] = df["Date"].dt.weekday
    df = df.join(df.groupby("SecuritiesCode").apply(holiday))
    df["Volume_ratio"] = df["Volume"]/df.groupby("SecuritiesCode")["Volume"].rolling(window=15, min_periods=1).mean().reset_index("SecuritiesCode",drop=True)
    df["Close_adj"] = df.groupby("SecuritiesCode").apply(lambda d:d["Close"]/d["AdjustmentFactor"].cumprod().shift().fillna(1)).reset_index("SecuritiesCode",drop=True)
    df[["MA1", "MA2", "MA_Cross"]] = df.groupby("SecuritiesCode").apply(lambda d: MAFeature()._create_feature(d.Close_adj))# .join(df["Target"].shift(-1)).groupby("MA_Cross").describe()
    df["Diff"] = (df["Close"] - df["Open"]) / df[["Close","Open"]].mean(axis=1)
    df["Diff_MA1"] = df["Close_adj"] - df["MA1"]
    df["Diff_MA2"] = df["Close_adj"] - df["MA2"]
    for i in range(1, 3):
        df["MA_Cross_lag_{:}".format(i)] = df.groupby("SecuritiesCode")["MA_Cross"].shift(i)

    df["DivMA"] = df.groupby("SecuritiesCode")["Close_adj"].apply(DMA)
    df["Div"] = df.groupby("SecuritiesCode")["Close_adj"].apply(divergence)
    df["Rsi"] = df.groupby("SecuritiesCode")["Close_adj"].apply(rsi)
    df = df.join(df.groupby("SecuritiesCode")["Close_adj"].apply(stochastic))
    
    
    return df

In [26]:
columns = [
    "Diff", "Close_adj","Volume_ratio",
    "before_holiday", "after_holiday",
    "Diff_MA1", "Diff_MA2","MA_Cross",
    'MA_Cross_lag_1', 'MA_Cross_lag_2',
    "DivMA", "Div", "Rsi", "%K", "FAST-%D","SLOW-%D",
]

##5. Building the Model

In [27]:
%%time
df = pd.read_csv("/content/drive/MyDrive/Especializacion BigData EIA/TrabajoFinal/Overview_And_InitialPredictions/supplemental_files/stock_prices.csv", parse_dates=["Date"])
df = make_features(df).join(df.Target)

CPU times: user 2min 18s, sys: 440 ms, total: 2min 18s
Wall time: 2min 18s


In [28]:
df

,Date,SecuritiesCode,Open,Close,AdjustmentFactor,Volume,weekday,before_holiday,after_holiday,Volume_ratio,...,Diff_MA2,MA_Cross_lag_1,MA_Cross_lag_2,DivMA,Div,Rsi,%K,FAST-%D,SLOW-%D,Target
0,2021-12-06,1301,2982.0,2971.0,1.0,8900,0,False,True,1.000000,...,0.00,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-0.003263
1,2021-12-06,1332,592.0,589.0,1.0,1360800,0,False,True,1.000000,...,0.00,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-0.008993
2,2021-12-06,1333,2368.0,2377.0,1.0,125900,0,False,True,1.000000,...,0.00,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-0.009963
3,2021-12-06,1375,1230.0,1224.0,1.0,81100,0,False,True,1.000000,...,0.00,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-0.015032
4,2021-12-06,1376,1339.0,1351.0,1.0,6200,0,False,True,1.000000,...,0.00,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.002867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269876,2022-06-24,9990,576.0,564.0,1.0,24200,4,True,False,0.542277,...,-4.44,0.0,0.0,-0.007811,-0.204258,0.411392,0.615385,0.535897,0.438095,0.027073
269877,2022-06-24,9991,810.0,815.0,1.0,8700,4,True,False,0.493570,...,-8.68,0.0,0.0,-0.010538,-0.489340,0.428571,0.379310,0.298851,0.262138,0.001220
269878,2022-06-24,9993,1548.0,1497.0,1.0,12600,4,True,False,2.816692,...,-15.84,1.0,0.0,-0.010470,-1.114802,0.469136,0.000000,0.595960,0.626263,0.001329
269879,2022-06-24,9994,2507.0,2527.0,1.0,7300,4,True,False,1.411082,...,38.68,0.0,1.0,0.015545,1.392220,0.537736,1.000000,0.657658,0.552553,0.003185


In [29]:
def train_model(X, y):
#     fast
    model=LGBMRegressor(boosting_type="dart",
                        num_leaves=31,max_depth=12,
                        learning_rate=0.2, n_estimators=100,
                        random_state=0)
#     model=LGBMRegressor(boosting_type="dart",
#                     num_leaves=31,max_depth=12,
#                     learning_rate=0.02, n_estimators=1000,
#                     random_state=0)
    model.fit(X,y)
    # model.score(X,y)
    return model

In [30]:
%%time
models = {}
num = 0
for code, d in df.groupby("SecuritiesCode"):
    d = d[~d.Target.isnull()]
    X = d[columns]
    y = d.Target
    model = train_model(X, y)
    models[code] = model

CPU times: user 1min 34s, sys: 5.88 s, total: 1min 40s
Wall time: 51.7 s


In [31]:
import warnings, gc
import numpy as np 
import pandas as pd
import matplotlib.colors
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
# from datetime import datetime, timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error,mean_absolute_error
from lightgbm import LGBMRegressor
from decimal import ROUND_HALF_UP, Decimal
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff

In [32]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
#         weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        df_len = len(df)
        if df_len>=portfolio_size:
            weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
            purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        else:
            weights = np.linspace(start=toprank_weight_ratio, stop=1, num=df_len)
            purchase = (df.sort_values(by='Rank')['Target'][:df_len] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [33]:
%%time
ts_fold = TimeSeriesSplit(n_splits=5, gap=0)
df_input = df.copy().sort_values(['Date','SecuritiesCode'])
# prices=price_features.dropna().sort_values(['Date','SecuritiesCode'])
y=df_input['Target'].to_numpy()
X=df_input.drop(['Target'],axis=1)

feat_importance=pd.DataFrame()
sharpe_ratio=[]
result = {}
    
for fold, (train_idx, val_idx) in enumerate(ts_fold.split(X, y)):
    
    print("\n========================== Fold {} ==========================".format(fold+1))
    df_train = df_input.iloc[train_idx,:]
    df_valid = df_input.iloc[val_idx,:]
    X_train, y_train = X.iloc[train_idx,:], y[train_idx]
    X_valid, y_val = X.iloc[val_idx,:], y[val_idx]
    
    print("Train Date range: {} to {}".format(X_train.Date.min(),X_train.Date.max()))
    print("Valid Date range: {} to {}".format(X_valid.Date.min(),X_valid.Date.max()))
    
#     X_train.drop(['Date','SecuritiesCode'], axis=1, inplace=True)
#     X_train.drop(['Date'], axis=1, inplace=True)
#     X_val=X_valid[X_valid.columns[~X_valid.columns.isin(['Date','SecuritiesCode'])]]
#     val_dates=X_valid.Date.unique()[1:-1]
#     print("\nTrain Shape: {} {}, Valid Shape: {} {}".format(X_train.shape, y_train.shape, X_val.shape, y_val.shape))
    
#     params = {'n_estimators': 1000,
#               'num_leaves' : 31,
#               'max_depth' : 12,
#               'learning_rate': 0.02,
#               'colsample_bytree': 0.9,
#               'subsample': 0.8,
#               'reg_alpha': 0.4,
#               'metric': 'mae',
#               'random_state': 0,
#               'boosting_type': "dart"
#               }

#     gbm = LGBMRegressor(**params).fit(X_train, y_train, 
#                                       eval_set=[(X_train, y_train), (X_val, y_val)],
#                                       verbose=300, 
#                                       eval_metric=['mae','mse'])

    models = {}
    for code, d in df_train.groupby("SecuritiesCode"):
        X_tmp = d[columns]
        y_tmp = d.Target
        model = train_model(X_tmp[~y_tmp.isnull()], y_tmp[~y_tmp.isnull()])
        models[code] = model
        
#     gbm = train_model(X_train, y_train)
#     y_pred = gbm.predict(X_val)
#     rmse = np.sqrt(mean_squared_error(y_val, y_pred))
#     mae = mean_absolute_error(y_val, y_pred)
#     feat_importance["Importance_Fold"+str(fold)]=gbm.feature_importances_
#     feat_importance.set_index(X_train.columns, inplace=True)
    
    rank=[]
    df_valid_tmp = df_valid.copy()
#     X_val_df=X_valid[X_valid.Date.isin(val_dates)]
    for i, d in df_valid_tmp.groupby("SecuritiesCode"):
        df_valid_tmp.loc[d.index, "pred"]=models[i].predict(d[columns])
    for date, d in df_valid_tmp.groupby("Date"):
        df_valid_tmp.loc[d.index, "Rank"]= (d.pred.rank(method="first", ascending=False)-1).astype(int)
#     val_df_tmp = X_val_df.merge(y_val)
    sharpe=calc_spread_return_sharpe(df_valid_tmp)
    result[fold] =  df_valid_tmp
    
#     for i in X_val_df.Date.unique():
#         tmp_result_list = []
#         code_list = []
#         for i_code in X_val_df[(X_val_df.Date == i)].SecuritiesCode.unique():
#             temp_df = X_val_df[(X_val_df.Date == i)&(X_val_df.SecuritiesCode == i_code)].drop(['Date','SecuritiesCode'],axis=1)
#             tmp_result = models[code].predict(temp_df[columns])
#             tmp_result_list.append(tmp_result)
#             code_list.append(code_list)
#         temp_df = pd.DataFrame({"pred":tmp_result_list, })
#         temp_df["Rank"] = (temp_df["pred"].rank(method="first", ascending=False)-1).astype(int)
#         rank.append(temp_df["Rank"].values)

#     stock_rank=pd.Series([x for y in rank for x in y], name="Rank")
#     df_tmp=pd.concat([X_val_df.reset_index(drop=True),stock_rank,
#                   prices[prices.Date.isin(val_dates)]['Target'].reset_index(drop=True)], axis=1)
#     sharpe=calc_spread_return_sharpe(df_tmp)
    sharpe_ratio.append(sharpe)
    print("Valid Sharpe: {}".format(sharpe))
    
    
#     del X_train, y_train,  X_val, y_val
#     gc.collect()
    
print("\nAverage cross-validation Sharpe Ratio: {:.4f}, standard deviation = {:.2f}.".format(np.mean(sharpe_ratio),np.std(sharpe_ratio)))


========================== Fold 1 ==========================
Train Date range: 2021-12-06 00:00:00 to 2022-01-07 00:00:00
Valid Date range: 2022-01-07 00:00:00 to 2022-02-09 00:00:00
Valid Sharpe: 0.14301861223181092

========================== Fold 2 ==========================
Train Date range: 2021-12-06 00:00:00 to 2022-02-09 00:00:00
Valid Date range: 2022-02-09 00:00:00 to 2022-03-16 00:00:00
Valid Sharpe: 0.051756526656379254

========================== Fold 3 ==========================
Train Date range: 2021-12-06 00:00:00 to 2022-03-16 00:00:00
Valid Date range: 2022-03-16 00:00:00 to 2022-04-18 00:00:00
Valid Sharpe: 0.04401343400476566

========================== Fold 4 ==========================
Train Date range: 2021-12-06 00:00:00 to 2022-04-18 00:00:00
Valid Date range: 2022-04-18 00:00:00 to 2022-05-25 00:00:00
Valid Sharpe: -0.0027267208254289355

========================== Fold 5 ==========================
Train Date range: 2021-12-06 00:00:00 to 2022-05-25 00:00:00
V